# 🚀 Entrenamiento YOLOv8 en Google Colab

Este notebook entrena un modelo YOLOv8 con Ultralytics en Google Colab para detección de vida silvestre aérea.

## 📋 Características
- **Modelo**: YOLOv8s, YOLOv8m, YOLOv8l, YOLOv8x
- **Detección automática de GPU**
- **Conversión automática COCO a YOLO**
- **Visualización de resultados**
- **Exportación a ONNX/TorchScript**
- **Análisis de métricas detallado**

## 🎯 Clases de Animales
- Buffalo
- Elephant  
- Kob
- Alcelaphinae
- Warthog
- Waterbuck

## ⚙️ Configuración por Defecto
- **Modelo**: YOLOv8s (balanceado entre velocidad y precisión)
- **Tamaño de imagen**: 640x640
- **Épocas**: 100
- **Batch size**: 16
- **Optimizador**: AdamW
- **Mixed Precision**: Habilitado

## 🔬 Ventajas de YOLOv8
- **Rápido**: Entrenamiento e inferencia eficientes
- **Preciso**: Mejor rendimiento que versiones anteriores
- **Fácil de usar**: API simple de Ultralytics
- **Flexible**: Múltiples tamaños de modelo

## 🔧 Instalación de Dependencias

In [ ]:
# Instalar dependencias
%pip install -q ultralytics pyyaml opencv-python pillow tqdm matplotlib seaborn pandas

# Verificar instalación
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 📦 Importar Librerías

In [ ]:
import os
import sys
import yaml
import torch
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
import shutil
import json
from tqdm import tqdm

from ultralytics import YOLO
from google.colab import files, drive
from IPython.display import Image as IPImage, display

# Configurar matplotlib
plt.style.use('default')
sns.set_palette("husl")

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 📁 Configuración de Datos

In [ ]:
# ============================================================
# CONFIGURACIÓN DE RUTAS PRINCIPALES (Unificado con HerdNet)
# ============================================================

# Definir la ruta base principal (ajustar según el entorno: Drive o local)
BASE_DIR = Path("/content/drive/MyDrive/aerial-wildlife-count")

# ============================================================
# RUTAS DE IMÁGENES Y ANOTACIONES (COCO JSON) - Igual que HerdNet
# ============================================================

# Rutas de los archivos de anotaciones en formato COCO
TRAIN_ANN_FILE = BASE_DIR / "data" / "coco" / "train" / "train_annotations.json"
VAL_ANN_FILE = BASE_DIR / "data" / "coco" / "val" / "val_annotations.json"
TEST_ANN_FILE = BASE_DIR / "data" / "coco" / "test" / "test_annotations.json"

# Rutas de las carpetas de imágenes correspondientes a cada conjunto
TRAIN_IMG_DIR = BASE_DIR / "data" / "images" / "train"
VAL_IMG_DIR = BASE_DIR / "data" / "images" / "val"
TEST_IMG_DIR = BASE_DIR / "data" / "images" / "test"

# ============================================================
# RUTAS ALTERNATIVAS (Fallback para compatibilidad)
# ============================================================

# Rutas alternativas si no existe la estructura estándar
TRAIN_ANN_FILE_ALT = BASE_DIR / "data" / "groundtruth" / "json" / "big_size" / "train_big_size_A_B_E_K_WH_WB.json"
VAL_ANN_FILE_ALT = BASE_DIR / "data" / "groundtruth" / "json" / "big_size" / "val_big_size_A_B_E_K_WH_WB.json"
TEST_ANN_FILE_ALT = BASE_DIR / "data" / "groundtruth" / "json" / "big_size" / "test_big_size_A_B_E_K_WH_WB.json"

# Rutas alternativas para imágenes
TRAIN_IMG_DIR_ALT = BASE_DIR / "data" / "train"
VAL_IMG_DIR_ALT = BASE_DIR / "data" / "val"
TEST_IMG_DIR_ALT = BASE_DIR / "data" / "test"

# ============================================================
# FUNCIÓN PARA CONFIGURAR DATOS (Unificada con HerdNet)
# ============================================================

def setup_data():
    """Configurar datos desde Google Drive con detección automática de estructura"""
    
    # Montar Google Drive
    drive.mount('/content/drive')
    print("✅ Google Drive montado")
    
    # Verificar si existe la estructura estándar (igual que HerdNet)
    if TRAIN_ANN_FILE.exists() and TRAIN_IMG_DIR.exists():
        print("✅ Datos encontrados (estructura estándar COCO)")
        return str(BASE_DIR), "standard"
    elif TRAIN_ANN_FILE_ALT.exists() and TRAIN_IMG_DIR_ALT.exists():
        print("✅ Datos encontrados (estructura alternativa groundtruth)")
        return str(BASE_DIR), "groundtruth"
    else:
        # Buscar en ubicaciones alternativas
        drive_path = "/content/drive/MyDrive"
        possible_paths = [
            f"{drive_path}/aerial-wildlife-count",
            f"{drive_path}/datasets/aerial-wildlife-count",
            f"{drive_path}/Colab Notebooks/aerial-wildlife-count"
        ]
        
        for path in possible_paths:
            if os.path.exists(path):
                print(f"✅ Dataset encontrado en ubicación alternativa: {path}")
                return path, "legacy"
        
        print("❌ No se encontró el dataset en Google Drive")
        return None, None

# Configurar datos
data_path, data_type = setup_data()
if data_path:
    print(f"📁 Ruta de datos configurada: {data_path}")
    print(f"📊 Tipo de datos: {data_type}")
else:
    print("⚠️  Configura los datos manualmente antes de continuar")

## 📊 Análisis de Datos


In [ ]:
# Función para encontrar y analizar datasets (Unificada con HerdNet)
def find_and_analyze_datasets():
    """Encontrar y analizar datasets disponibles según el tipo de datos"""
    
    if not data_path:
        print("❌ No hay ruta de datos configurada")
        return []
    
    datasets_found = []
    
    if data_type == "standard":
        # Estructura estándar COCO (igual que HerdNet)
        datasets_found = [TRAIN_ANN_FILE, VAL_ANN_FILE, TEST_ANN_FILE]
        print("📊 Usando datasets estándar COCO:")
        print(f"  Train: {TRAIN_ANN_FILE}")
        print(f"  Val: {VAL_ANN_FILE}")
        print(f"  Test: {TEST_ANN_FILE}")
        
    elif data_type == "groundtruth":
        # Estructura alternativa groundtruth
        datasets_found = [TRAIN_ANN_FILE_ALT, VAL_ANN_FILE_ALT, TEST_ANN_FILE_ALT]
        print("📊 Usando datasets groundtruth:")
        print(f"  Train: {TRAIN_ANN_FILE_ALT}")
        print(f"  Val: {VAL_ANN_FILE_ALT}")
        print(f"  Test: {TEST_ANN_FILE_ALT}")
        
    else:
        # Estructura legacy - buscar archivos JSON
        data_root = Path(data_path)
        json_patterns = [
            "**/train_*.json",
            "**/val_*.json", 
            "**/test_*.json",
            "**/*_big_size_*.json",
            "**/*_subframes_*.json"
        ]
        
        for pattern in json_patterns:
            for json_file in data_root.glob(pattern):
                if json_file.is_file():
                    datasets_found.append(json_file)
        
        print(f"📊 Datasets encontrados en estructura legacy ({len(datasets_found)}):")
        for i, dataset in enumerate(datasets_found):
            print(f"  {i+1}. {dataset}")
    
    # Analizar el primer dataset encontrado
    if datasets_found:
        sample_data = analyze_dataset(datasets_found[0])
        return datasets_found
    
    return []

# Función para analizar un dataset COCO
def analyze_dataset(json_path):
    """Analizar estadísticas de un dataset COCO"""
    
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    print(f"\n📈 Análisis de {json_path.name}:")
    print(f"  Imágenes: {len(data['images'])}")
    print(f"  Anotaciones: {len(data['annotations'])}")
    print(f"  Categorías: {len(data['categories'])}")
    
    # Estadísticas por categoría
    cat_counts = {}
    for ann in data['annotations']:
        cat_id = ann['category_id']
        cat_counts[cat_id] = cat_counts.get(cat_id, 0) + 1
    
    print("\n  Distribución por categoría:")
    for cat in data['categories']:
        count = cat_counts.get(cat['id'], 0)
        print(f"    {cat['name']}: {count}")
    
    return data

# Buscar y analizar datasets
datasets = find_and_analyze_datasets()


## ⚙️ Configuración del Entrenamiento

In [ ]:
# Configuración del entrenamiento
class YOLOConfig:
    def __init__(self):
        # Parámetros del modelo
        self.model = 'yolov8s.pt'  # yolov8n.pt, yolov8s.pt, yolov8m.pt, yolov8l.pt, yolov8x.pt
        self.image_size = 640
        self.epochs = 100
        self.batch_size = 16  # Ajustar según memoria disponible
        self.learning_rate = 0.01
        self.patience = 10
        self.device = 0  # GPU 0
        self.workers = 4
        self.project = '/content/runs_yolo'
        self.name = 'yolo_aerial_wildlife'
        self.fp16 = True  # Mixed precision
        
        # Clases del dataset
        self.classes = [
            "Buffalo", "Elephant", "Kob", 
            "Alcelaphinae", "Warthog", "Waterbuck"
        ]
        
    def print_config(self):
        print("🔧 Configuración YOLOv8:")
        print(f"  Modelo: {self.model}")
        print(f"  Tamaño de imagen: {self.image_size}")
        print(f"  Épocas: {self.epochs}")
        print(f"  Batch size: {self.batch_size}")
        print(f"  Learning rate: {self.learning_rate}")
        print(f"  Dispositivo: {self.device}")
        print(f"  Mixed precision: {self.fp16}")

# Crear instancia de configuración
yolo_config = YOLOConfig()
yolo_config.print_config()

## 🔄 Conversión de Datos COCO a YOLO


In [ ]:
# Función para convertir COCO a YOLO
def coco_to_yolo(coco_json_path, images_dir, output_dir, class_names):
    """Convierte anotaciones COCO a formato YOLO"""
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Leer archivo COCO
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)
    
    # Crear mapeo de categorías
    cat_id_to_class = {cat['id']: cat['name'] for cat in coco_data['categories']}
    class_to_id = {name: idx for idx, name in enumerate(class_names)}
    
    # Crear mapeo de imágenes
    img_id_to_info = {img['id']: img for img in coco_data['images']}
    
    # Procesar anotaciones
    annotations_by_image = {}
    for ann in coco_data['annotations']:
        img_id = ann['image_id']
        if img_id not in annotations_by_image:
            annotations_by_image[img_id] = []
        annotations_by_image[img_id].append(ann)
    
    # Convertir cada imagen
    for img_id, img_info in tqdm(img_id_to_info.items(), desc="Convirtiendo a YOLO"):
        img_name = img_info['file_name']
        img_width = img_info['width']
        img_height = img_info['height']
        
        # Crear archivo de anotación YOLO
        txt_name = Path(img_name).stem + '.txt'
        txt_path = output_dir / txt_name
        
        with open(txt_path, 'w') as f:
            if img_id in annotations_by_image:
                for ann in annotations_by_image[img_id]:
                    cat_name = cat_id_to_class[ann['category_id']]
                    if cat_name in class_to_id:
                        class_id = class_to_id[cat_name]
                        
                        # Convertir bbox [x, y, w, h] a [center_x, center_y, w, h] normalizado
                        x, y, w, h = ann['bbox']
                        center_x = (x + w/2) / img_width
                        center_y = (y + h/2) / img_height
                        norm_w = w / img_width
                        norm_h = h / img_height
                        
                        f.write(f"{class_id} {center_x:.6f} {center_y:.6f} {norm_w:.6f} {norm_h:.6f}\n")
    
    print(f"✅ Convertido {len(img_id_to_info)} imágenes a formato YOLO")
    return output_dir

# Preparar datos para conversión (Unificado con HerdNet)
def prepare_yolo_data():
    """Preparar datos para entrenamiento YOLO según el tipo de datos"""
    
    if data_type == "standard":
        # Usar rutas estándar COCO (igual que HerdNet)
        train_json = TRAIN_ANN_FILE
        val_json = VAL_ANN_FILE
        test_json = TEST_ANN_FILE
        
        # Verificar que existan
        if not all([train_json.exists(), val_json.exists()]):
            print("❌ Faltan archivos de datos estándar COCO")
            return None, None, None
            
    elif data_type == "groundtruth":
        # Usar rutas groundtruth
        train_json = TRAIN_ANN_FILE_ALT
        val_json = VAL_ANN_FILE_ALT
        test_json = TEST_ANN_FILE_ALT
        
        # Verificar que existan
        if not all([train_json.exists(), val_json.exists()]):
            print("❌ Faltan archivos de datos groundtruth")
            return None, None, None
            
    else:
        # Estructura legacy - buscar en datasets encontrados
        if not datasets:
            print("❌ No hay datasets disponibles")
            return None, None, None
        
        # Buscar archivos de datos
        train_files = [d for d in datasets if 'train' in d.name.lower()]
        val_files = [d for d in datasets if 'val' in d.name.lower()]
        test_files = [d for d in datasets if 'test' in d.name.lower()]
        
        train_json = train_files[0] if train_files else None
        val_json = val_files[0] if val_files else None
        test_json = test_files[0] if test_files else None
        
        if not all([train_json, val_json]):
            print("❌ Faltan archivos de datos. Necesitas al menos train y val JSON files.")
            return None, None, None
    
    print(f"✅ Archivos seleccionados:")
    print(f"  Train: {train_json}")
    print(f"  Val: {val_json}")
    if test_json and test_json.exists():
        print(f"  Test: {test_json}")
    
    return train_json, val_json, test_json

# Preparar datos
train_json, val_json, test_json = prepare_yolo_data()

if train_json and val_json:
    print("✅ Datos preparados para conversión")
else:
    print("❌ Error preparando datos")


In [ ]:
# Convertir datos si están disponibles (Adaptado para pipelines)
if train_json and val_json:
    
    # Determinar directorios de imágenes según el tipo de datos
    if data_type == "standard":
        train_img_dir = TRAIN_IMG_DIR
        val_img_dir = VAL_IMG_DIR
        test_img_dir = TEST_IMG_DIR
    elif data_type == "groundtruth":
        train_img_dir = TRAIN_IMG_DIR_ALT
        val_img_dir = VAL_IMG_DIR_ALT
        test_img_dir = TEST_IMG_DIR_ALT
    else:
        # Estructura legacy
        train_img_dir = train_json.parent / "images" if (train_json.parent / "images").exists() else train_json.parent.parent / "train"
        val_img_dir = val_json.parent.parent / "val"
        test_img_dir = None
    
    # Convertir datos de entrenamiento
    print("🔄 Convirtiendo datos de entrenamiento...")
    train_yolo_dir = coco_to_yolo(
        train_json,
        train_img_dir,
        '/content/yolo_data/train/labels',
        yolo_config.classes
    )
    
    # Convertir datos de validación
    print("🔄 Convirtiendo datos de validación...")
    val_yolo_dir = coco_to_yolo(
        val_json,
        val_img_dir,
        '/content/yolo_data/val/labels',
        yolo_config.classes
    )
    
    # Crear directorios de imágenes
    os.makedirs('/content/yolo_data/train/images', exist_ok=True)
    os.makedirs('/content/yolo_data/val/images', exist_ok=True)
    
    # Copiar imágenes
    print("📁 Copiando imágenes...")
    for img_file in tqdm(os.listdir(train_img_dir)):
        if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            src = os.path.join(train_img_dir, img_file)
            dst = os.path.join('/content/yolo_data/train/images', img_file)
            if os.path.exists(src):
                shutil.copy2(src, dst)
    
    for img_file in tqdm(os.listdir(val_img_dir)):
        if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            src = os.path.join(val_img_dir, img_file)
            dst = os.path.join('/content/yolo_data/val/images', img_file)
            if os.path.exists(src):
                shutil.copy2(src, dst)
    
    # Copiar imágenes de test si están disponibles
    if test_img_dir and test_img_dir.exists():
        print("📁 Copiando imágenes de test...")
        os.makedirs('/content/yolo_data/test/images', exist_ok=True)
        for img_file in tqdm(os.listdir(test_img_dir)):
            if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                src = os.path.join(test_img_dir, img_file)
                dst = os.path.join('/content/yolo_data/test/images', img_file)
                if os.path.exists(src):
                    shutil.copy2(src, dst)
    
    # Crear archivo de configuración YOLO
    test_line = "test: test/images\n" if test_img_dir and test_img_dir.exists() else ""
    yolo_config_content = f"""# Dataset configuration for YOLOv8
path: /content/yolo_data
train: train/images
val: val/images
{test_line}
# Classes
nc: {len(yolo_config.classes)}
names: {yolo_config.classes}
"""
    
    with open('/content/yolo_data/dataset.yaml', 'w') as f:
        f.write(yolo_config_content)
    
    print("✅ Conversión completada")
    print(f"📊 Imágenes de entrenamiento: {len(os.listdir('/content/yolo_data/train/images'))}")
    print(f"📊 Imágenes de validación: {len(os.listdir('/content/yolo_data/val/images'))}")
    if test_img_dir and test_img_dir.exists():
        print(f"📊 Imágenes de test: {len(os.listdir('/content/yolo_data/test/images'))}")
    
else:
    print("❌ No se puede convertir sin datos válidos")


## 🚀 Entrenamiento del Modelo


In [ ]:
# Entrenar modelo YOLOv8
if train_json and val_json:
    
    print("🚀 Iniciando entrenamiento YOLOv8...")
    
    # Inicializar modelo
    model = YOLO(yolo_config.model)
    
    # Configurar parámetros de entrenamiento
    train_args = {
        'data': '/content/yolo_data/dataset.yaml',
        'epochs': yolo_config.epochs,
        'imgsz': yolo_config.image_size,
        'batch': yolo_config.batch_size,
        'device': yolo_config.device,
        'workers': yolo_config.workers,
        'project': yolo_config.project,
        'name': yolo_config.name,
        'patience': yolo_config.patience,
        'lr0': yolo_config.learning_rate,
        'amp': yolo_config.fp16,
        'save': True,
        'save_period': 10,
        'val': True,
        'plots': True,
        'verbose': True,
    }
    
    print("📋 Parámetros de entrenamiento:")
    for key, value in train_args.items():
        print(f"  {key}: {value}")
    
    # Iniciar entrenamiento
    results = model.train(**train_args)
    
    print("✅ Entrenamiento completado!")
    print(f"📁 Resultados guardados en: {yolo_config.project}/{yolo_config.name}")
    
else:
    print("❌ No se puede entrenar sin datos válidos")


## 🎉 ¡Entrenamiento Completado!

### 📋 Resumen del Entrenamiento
- **Modelo**: YOLOv8 con configuración {yolo_config.model}
- **Tamaño de imagen**: {yolo_config.image_size}x{yolo_config.image_size}
- **Épocas**: {yolo_config.epochs}
- **Clases detectadas**: {len(yolo_config.classes)} especies de animales

### 📊 Próximos Pasos
1. **Evaluar métricas**: Revisar mAP, precision, recall
2. **Ajustar hiperparámetros**: Si es necesario mejorar el rendimiento
3. **Exportar modelo**: Convertir a ONNX o TorchScript para deployment
4. **Probar en nuevas imágenes**: Validar en datos no vistos

### 🔧 Configuración Personalizada
Para modificar parámetros, edita la clase `YOLOConfig` en la celda de configuración:
- Cambiar modelo: `"yolov8s.pt"`, `"yolov8m.pt"`, `"yolov8l.pt"`, `"yolov8x.pt"`
- Ajustar épocas: `epochs = 200`
- Modificar tamaño de imagen: `image_size = 1024`
- Cambiar batch size: `batch_size = 32`

### 📚 Recursos Adicionales
- [Documentación Ultralytics](https://docs.ultralytics.com/)
- [YOLOv8 Paper](https://arxiv.org/abs/2305.09972)
- [GitHub Ultralytics](https://github.com/ultralytics/ultralytics)


## 📊 Visualización de Resultados

In [ ]:
# Cargar el mejor modelo entrenado
best_model_path = f"{yolo_config.project}/{yolo_config.name}/weights/best.pt"
model = YOLO(best_model_path)

# Visualizar curvas de entrenamiento
results_dir = f"{yolo_config.project}/{yolo_config.name}"
if os.path.exists(f"{results_dir}/results.png"):
    display(IPImage(f"{results_dir}/results.png"))

# Mostrar métricas finales
if os.path.exists(f"{results_dir}/results.csv"):
    results_df = pd.read_csv(f"{results_dir}/results.csv")
    print("📊 Métricas de entrenamiento:")
    print(results_df.tail(10))

# Mostrar matriz de confusión
if os.path.exists(f"{results_dir}/confusion_matrix.png"):
    print("\n📊 Matriz de Confusión:")
    display(IPImage(f"{results_dir}/confusion_matrix.png"))

## 🔍 Inferencia y Pruebas

In [ ]:
# Realizar inferencia en imágenes de prueba
if test_json and test_json.exists():
    # Determinar directorio de imágenes de test según el tipo de datos
    if data_type == "standard":
        test_images_dir = TEST_IMG_DIR
    elif data_type == "groundtruth":
        test_images_dir = TEST_IMG_DIR_ALT
    else:
        # Estructura legacy
        test_images_dir = test_json.parent.parent / "test"
    
    if test_images_dir and test_images_dir.exists():
        test_images = [f for f in os.listdir(test_images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
        # Seleccionar algunas imágenes para prueba
        sample_images = test_images[:5]  # Primeras 5 imágenes
        
        print(f"🔍 Realizando inferencia en {len(sample_images)} imágenes de prueba...")
        
        for img_name in sample_images:
            img_path = os.path.join(test_images_dir, img_name)
            
            # Realizar predicción
            results = model(img_path, conf=0.5)
            
            # Mostrar resultado
            for r in results:
                # Guardar imagen con predicciones
                output_path = f"/content/test_results_{img_name}"
                r.save(output_path)
                
                # Mostrar imagen
                display(IPImage(output_path))
                
                # Mostrar estadísticas
                print(f"📊 {img_name}: {len(r.boxes)} objetos detectados")
                if len(r.boxes) > 0:
                    for box in r.boxes:
                        class_id = int(box.cls[0])
                        confidence = float(box.conf[0])
                        class_name = yolo_config.classes[class_id]
                        print(f"  - {class_name}: {confidence:.2f}")
                print()
    else:
        print("⚠️  No se encontró directorio de imágenes de test")
else:
    print("⚠️  No hay datos de test disponibles para inferencia")

## 💾 Guardar y Exportar Modelo

In [ ]:
# Exportar modelo a ONNX para deployment
print("🔄 Exportando modelo a ONNX...")
onnx_path = model.export(format='onnx', imgsz=yolo_config.image_size)
print(f"✅ Modelo exportado a: {onnx_path}")

# Copiar resultados a Google Drive
drive_results_dir = f"/content/drive/MyDrive/aerial-wildlife-count/results/yolov8_{yolo_config.name}"
os.makedirs(drive_results_dir, exist_ok=True)

# Copiar archivos importantes
files_to_copy = [
    f"{results_dir}/weights/best.pt",
    f"{results_dir}/weights/last.pt",
    f"{results_dir}/results.png",
    f"{results_dir}/confusion_matrix.png",
    f"{results_dir}/results.csv",
    onnx_path
]

for file_path in files_to_copy:
    if os.path.exists(file_path):
        filename = os.path.basename(file_path)
        shutil.copy2(file_path, os.path.join(drive_results_dir, filename))
        print(f"📁 Copiado: {filename}")

print(f"✅ Resultados guardados en Google Drive: {drive_results_dir}")

# Mostrar resumen final
print("\n🎉 RESUMEN DEL ENTRENAMIENTO")
print("=" * 50)
print(f"Modelo: {yolo_config.model}")
print(f"Épocas: {yolo_config.epochs}")
print(f"Tamaño de imagen: {yolo_config.image_size}")
print(f"Batch size: {yolo_config.batch_size}")
print(f"Clases: {yolo_config.classes}")
print(f"Mejor modelo: {best_model_path}")
print(f"Modelo ONNX: {onnx_path}")
print(f"Resultados en Drive: {drive_results_dir}")